<a href="https://colab.research.google.com/github/wangyiyang/RAG-Cookbook-Code/blob/main/ch03/colbert_reranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install sentence-transformers torch transformers faiss-cpu

In [3]:
"""
ColBERT重排序系统
实现精细化的Token级交互重排序算法
"""

import numpy as np
from typing import List, Dict, Tuple, Optional
import torch
import torch.nn.functional as F
from dataclasses import dataclass
import time


@dataclass
class ColBERTResult:
    """ColBERT重排序结果"""
    document_id: str
    content: str
    score: float
    token_scores: List[float]
    interaction_matrix: Optional[np.ndarray] = None


class MockColBERTEncoder:
    """模拟的ColBERT编码器（实际应用中使用预训练模型）"""

    def __init__(self, embedding_dim: int = 128):
        self.embedding_dim = embedding_dim
        # 模拟词汇表
        self.vocab = {}
        self.vocab_size = 30000

    def tokenize(self, text: str) -> List[str]:
        """简单的分词（实际应用中使用BERT分词器）"""
        import re
        # 中英文混合分词
        tokens = re.findall(r'[\u4e00-\u9fff]|[a-zA-Z]+|\d+', text.lower())
        return tokens[:512]  # 限制最大长度

    def encode_tokens(self, tokens: List[str]) -> np.ndarray:
        """编码tokens为向量（模拟实现）"""
        embeddings = []
        for token in tokens:
            # 简单的hash编码模拟
            token_hash = hash(token) % self.vocab_size
            # 生成随机但一致的embedding
            np.random.seed(token_hash)
            embedding = np.random.normal(0, 1, self.embedding_dim)
            embedding = embedding / np.linalg.norm(embedding)  # 归一化
            embeddings.append(embedding)

        return np.array(embeddings)


class ColBERTReRanker:
    """ColBERT重排序器"""

    def __init__(self, embedding_dim: int = 128):
        self.encoder = MockColBERTEncoder(embedding_dim)
        self.embedding_dim = embedding_dim
        self.cache = {}  # 文档编码缓存

    def encode_query(self, query: str) -> np.ndarray:
        """编码查询为token embeddings"""
        tokens = self.encoder.tokenize(query)
        if not tokens:
            return np.array([]).reshape(0, self.embedding_dim)

        embeddings = self.encoder.encode_tokens(tokens)
        return embeddings

    def encode_document(self, document: str, doc_id: str = None) -> np.ndarray:
        """编码文档为token embeddings（带缓存）"""
        if doc_id and doc_id in self.cache:
            return self.cache[doc_id]

        tokens = self.encoder.tokenize(document)
        if not tokens:
            return np.array([]).reshape(0, self.embedding_dim)

        embeddings = self.encoder.encode_tokens(tokens)

        if doc_id:
            self.cache[doc_id] = embeddings

        return embeddings

    def compute_interaction_score(
        self,
        query_embeddings: np.ndarray,
        doc_embeddings: np.ndarray
    ) -> Tuple[float, np.ndarray]:
        """计算查询和文档的交互得分"""
        if query_embeddings.size == 0 or doc_embeddings.size == 0:
            return 0.0, np.array([])

        # 计算相似度矩阵：每个查询token与每个文档token的相似度
        similarity_matrix = np.dot(query_embeddings, doc_embeddings.T)

        # ColBERT核心：每个查询token找最相似的文档token
        max_similarities = np.max(similarity_matrix, axis=1)

        # 总分数：所有查询token的最大相似度之和
        total_score = np.sum(max_similarities)

        return total_score, max_similarities

    def rerank(
        self,
        query: str,
        documents: List[str],
        top_k: int = 5,
        doc_ids: Optional[List[str]] = None
    ) -> List[ColBERTResult]:
        """重排序文档"""
        if not documents:
            return []

        # 编码查询
        query_embeddings = self.encode_query(query)

        results = []

        for i, document in enumerate(documents):
            doc_id = doc_ids[i] if doc_ids else f"doc_{i}"

            # 编码文档
            doc_embeddings = self.encode_document(document, doc_id)

            # 计算交互得分
            score, token_scores = self.compute_interaction_score(
                query_embeddings, doc_embeddings
            )

            result = ColBERTResult(
                document_id=doc_id,
                content=document,
                score=score,
                token_scores=token_scores.tolist()
            )
            results.append(result)

        # 按分数排序
        results.sort(key=lambda x: x.score, reverse=True)

        return results[:top_k]

    def batch_rerank(
        self,
        queries: List[str],
        document_lists: List[List[str]],
        top_k: int = 5
    ) -> List[List[ColBERTResult]]:
        """批量重排序"""
        if len(queries) != len(document_lists):
            raise ValueError("查询和文档列表数量不匹配")

        results = []
        for query, docs in zip(queries, document_lists):
            query_results = self.rerank(query, docs, top_k)
            results.append(query_results)

        return results

    def explain_ranking(
        self,
        query: str,
        document: str
    ) -> Dict[str, any]:
        """解释排序结果"""
        query_tokens = self.encoder.tokenize(query)
        doc_tokens = self.encoder.tokenize(document)

        query_embeddings = self.encode_query(query)
        doc_embeddings = self.encode_document(document)

        score, token_scores = self.compute_interaction_score(
            query_embeddings, doc_embeddings
        )

        # 找到每个查询token的最佳匹配文档token
        similarity_matrix = np.dot(query_embeddings, doc_embeddings.T)
        best_matches = np.argmax(similarity_matrix, axis=1)

        explanations = []
        for i, (q_token, score) in enumerate(zip(query_tokens, token_scores)):
            if i < len(best_matches):
                best_doc_token = doc_tokens[best_matches[i]] if best_matches[i] < len(doc_tokens) else "N/A"
                explanations.append({
                    'query_token': q_token,
                    'matched_doc_token': best_doc_token,
                    'similarity_score': score
                })

        return {
            'total_score': score,
            'query_tokens': query_tokens,
            'doc_tokens': doc_tokens,
            'token_explanations': explanations,
            'avg_token_score': np.mean(token_scores) if len(token_scores) > 0 else 0
        }

    def get_cache_stats(self) -> Dict[str, int]:
        """获取缓存统计"""
        return {
            'cached_documents': len(self.cache),
            'cache_size_mb': sum(emb.nbytes for emb in self.cache.values()) / (1024 * 1024)
        }

    def clear_cache(self) -> None:
        """清空缓存"""
        self.cache.clear()


class ColBERTBatchProcessor:
    """ColBERT批处理器"""

    def __init__(self, reranker: ColBERTReRanker, batch_size: int = 32):
        self.reranker = reranker
        self.batch_size = batch_size

    def process_large_dataset(
        self,
        queries: List[str],
        document_collection: List[str],
        top_k_per_query: int = 10
    ) -> List[List[ColBERTResult]]:
        """处理大规模数据集"""
        results = []

        for i, query in enumerate(queries):
            print(f"处理查询 {i+1}/{len(queries)}: {query[:50]}...")

            # 分批处理文档
            query_results = []
            for j in range(0, len(document_collection), self.batch_size):
                batch_docs = document_collection[j:j + self.batch_size]
                batch_doc_ids = [f"doc_{j+k}" for k in range(len(batch_docs))]

                batch_results = self.reranker.rerank(
                    query, batch_docs,
                    top_k=len(batch_docs),  # 先获取所有结果
                    doc_ids=batch_doc_ids
                )
                query_results.extend(batch_results)

            # 重新排序并取top_k
            query_results.sort(key=lambda x: x.score, reverse=True)
            results.append(query_results[:top_k_per_query])

        return results


# 使用示例
if __name__ == "__main__":
    # 创建重排序器
    reranker = ColBERTReRanker()

    # 测试数据
    query = "RAG检索增强生成技术"
    documents = [
        "RAG是一种结合检索和生成的AI技术，能够从外部知识库中检索相关信息",
        "BERT是谷歌开发的预训练语言模型，在自然语言处理任务中表现优异",
        "检索增强生成技术通过外部知识提升语言模型的回答质量和准确性",
        "ColBERT是一种高效的检索模型，采用后期交互架构",
        "向量数据库存储文档的语义表示，支持相似性搜索"
    ]

    # 执行重排序
    print("🚀 ColBERT重排序测试")
    print("=" * 50)

    start_time = time.time()
    results = reranker.rerank(query, documents, top_k=3)
    elapsed = time.time() - start_time

    print(f"查询: {query}")
    print(f"处理时间: {elapsed:.3f}s")
    print()

    for i, result in enumerate(results, 1):
        print(f"{i}. [得分: {result.score:.3f}]")
        print(f"   文档: {result.content}")
        print(f"   Token平均分: {np.mean(result.token_scores):.3f}")
        print()

    # 排序解释
    print("📊 排序解释（Top 1）:")
    explanation = reranker.explain_ranking(query, results[0].content)
    print(f"总分: {explanation['total_score']:.3f}")
    print(f"平均Token分: {explanation['avg_token_score']:.3f}")
    print()
    print("Token匹配详情:")
    for exp in explanation['token_explanations'][:5]:  # 只显示前5个
        print(f"  '{exp['query_token']}' -> '{exp['matched_doc_token']}' (分数: {exp['similarity_score']:.3f})")

    # 缓存统计
    cache_stats = reranker.get_cache_stats()
    print(f"\n📈 缓存统计:")
    print(f"  缓存文档数: {cache_stats['cached_documents']}")
    print(f"  缓存大小: {cache_stats['cache_size_mb']:.2f}MB")

    # 批处理测试
    print("\n🔄 批处理测试:")
    batch_processor = ColBERTBatchProcessor(reranker, batch_size=3)

    test_queries = ["RAG技术", "BERT模型"]
    batch_results = batch_processor.process_large_dataset(
        test_queries, documents, top_k_per_query=2
    )

    for i, (query, results) in enumerate(zip(test_queries, batch_results)):
        print(f"\n查询 {i+1}: {query}")
        for j, result in enumerate(results):
            print(f"  {j+1}. [分数: {result.score:.3f}] {result.content[:40]}...")


🚀 ColBERT重排序测试
查询: RAG检索增强生成技术
处理时间: 0.076s

1. [得分: 8.169]
   文档: 检索增强生成技术通过外部知识提升语言模型的回答质量和准确性
   Token平均分: 0.908

2. [得分: 7.354]
   文档: RAG是一种结合检索和生成的AI技术，能够从外部知识库中检索相关信息
   Token平均分: 0.817

3. [得分: 3.362]
   文档: ColBERT是一种高效的检索模型，采用后期交互架构
   Token平均分: 0.374

📊 排序解释（Top 1）:
总分: 1.000
平均Token分: 0.908

Token匹配详情:
  'rag' -> '知' (分数: 0.169)
  '检' -> '检' (分数: 1.000)
  '索' -> '索' (分数: 1.000)
  '增' -> '增' (分数: 1.000)
  '强' -> '强' (分数: 1.000)

📈 缓存统计:
  缓存文档数: 5
  缓存大小: 0.12MB

🔄 批处理测试:
处理查询 1/2: RAG技术...
处理查询 2/2: BERT模型...

查询 1: RAG技术
  1. [分数: 3.000] RAG是一种结合检索和生成的AI技术，能够从外部知识库中检索相关信息...
  2. [分数: 2.169] 检索增强生成技术通过外部知识提升语言模型的回答质量和准确性...

查询 2: BERT模型
  1. [分数: 3.000] BERT是谷歌开发的预训练语言模型，在自然语言处理任务中表现优异...
  2. [分数: 2.207] ColBERT是一种高效的检索模型，采用后期交互架构...
